# 06 - Neural Network (TensorFlow/Keras)

Goal: train a simple neural network within this notebook (no CLI training).

Key elements:
- Input normalization (using scaler from preprocessing)
- Dropout regularization
- Early stopping
- Training history plots


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import matplotlib.pyplot as plt

PROCESSED_DIR = Path('data/processed')
X_train = pd.read_csv(PROCESSED_DIR / 'X_train_scaled.csv')
X_test = pd.read_csv(PROCESSED_DIR / 'X_test_scaled.csv')
y_train = pd.read_csv(PROCESSED_DIR / 'y_train.csv').squeeze()
y_test = pd.read_csv(PROCESSED_DIR / 'y_test.csv').squeeze()

# Build model
inputs = keras.Input(shape=(X_train.shape[1],))
x = keras.layers.Dense(64, activation='relu')(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.AUC(name='roc_auc'), keras.metrics.Recall(name='recall')])

callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_roc_auc')]
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=256,
    callbacks=callbacks,
    class_weight={0: 1.0, 1: (len(y_train)-y_train.sum())/y_train.sum()}
)

# Evaluation
metrics = model.evaluate(X_test, y_test, return_dict=True)
metrics

In [ ]:
# Plot training history
fig, ax = plt.subplots(1,2, figsize=(12,4))
ax[0].plot(history.history['roc_auc'], label='train AUC')
ax[0].plot(history.history['val_roc_auc'], label='val AUC')
ax[0].set_title('ROC-AUC')
ax[0].legend()

ax[1].plot(history.history['recall'], label='train recall')
ax[1].plot(history.history['val_recall'], label='val recall')
ax[1].set_title('Recall')
ax[1].legend()
plt.show()

Save the trained model to `models/saved_models` if performance is acceptable.